# Library

In [25]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
import time

In [28]:
# Open the file in read mode ('r')
with open('cookies.txt', 'r') as file:
    cookies_text = file.read()

# Now, evaluate the content to get the cookies dictionary
cookies = eval(cookies_text)

In [29]:
with open('headers.txt', 'r') as file:
    headers_text = file.read()

# Now, evaluate the content to get the cookies dictionary
headers = eval(headers_text)

In [59]:
with open('params.txt', 'r') as file:
    params_text = file.read()

# Now, evaluate the content to get the cookies dictionary
query_id = eval(params_text)

In [60]:
query_id['query_id']

'9957820854288654'

# Read Base Account

In [30]:
df_base_account = pd.read_csv('base_account.csv')
df_base_account.head()

,username
0,officialdonat_bahagia
1,sagara.homemade
2,dobul.donut
3,sera.treats
4,donusaurus


# Finding Account Id

In [31]:
for user in df_base_account['username']:
    print(user)

officialdonat_bahagia
sagara.homemade
dobul.donut
sera.treats
donusaurus
thedimsumco
dimsum.endut
dimsumbenhil
dapur.roja
dimsum_mr.san


In [32]:
def finding_base_account_id(username, cookies, headers):
    import requests
    
    cookies = cookies
    
    headers = headers
    
    params = {
        'username': username,
    }

    response = requests.get(
        'https://www.instagram.com/api/v1/users/web_profile_info/',
        params=params,
        cookies=cookies,
        headers=headers,
    )
    
    return json.loads(response.text)['data']['user']
    # return response


In [42]:
test = finding_base_account_id('officialdonat_bahagia', cookies, headers)
test

{'ai_agent_type': None,
 'biography': 'DONAT BIKIN BAHAGIA | 70 VARIANT RASA | BEBAS PILIH | DIBUAT DARI HATI OLEH PENGRAJIN KEBAHAGIAAN 💕 FRESHLY MADE FOR U . \nCEK LOKASI DLL DI LINK YA ⚘️',
 'bio_links': [{'title': '',
   'lynx_url': 'https://l.instagram.com/?u=http%3A%2F%2Fwww.donatbahagia.id%2F&e=AT1lGl_xObm99kZefMHqBkIqdNIuPlVBRO1_W0RPxAuMH4fkx4xBKXCvIWHcX0AiHt338eEmMnupcHfiH93hw9vUjW880QQyT1o8yd9m3t_EIQskTcF3RQ',
   'url': 'http://www.donatbahagia.id',
   'link_type': 'external'},
  {'title': 'whatsapp dobamin dobah',
   'lynx_url': 'https://l.instagram.com/?u=https%3A%2F%2Fwa.me%2Fmessage%2F2HND7WUWT4UCL1&e=AT12laLwSlJiVaimL0jN6Pp-BQrIEsx73_H-3DGYiIdB_aXHrOSSI3a2YZomlm3ac3ktV4tdcwLRia7dt6ZavMdoLd4oQCErP_r-vYFkgSmx8fhdz3BoMg',
   'url': 'https://wa.me/message/2HND7WUWT4UCL1',
   'link_type': 'external'}],
 'fb_profile_biolink': None,
 'biography_with_entities': {'raw_text': 'DONAT BIKIN BAHAGIA | 70 VARIANT RASA | BEBAS PILIH | DIBUAT DARI HATI OLEH PENGRAJIN KEBAHAGIAAN 💕 FRESH

In [48]:
from tqdm import tqdm
import time

base_account_list = []

base_account_col = ['id'
                    , 'username'
                    , 'full_name'
                    , 'biography_desc'
                    , 'category_name'
                    , 'followers'
                    , 'external_url'
                    , 'business_email'
                    , 'business_phone_number'        
                   ]

for user in tqdm(df_base_account['username'], desc="Processing users"):
    try:
        temp = finding_base_account_id(user, cookies, headers)
        base_account_list.append(
            [
                temp['id']
                , temp['username']
                , temp['full_name']
                , temp['biography_with_entities']['raw_text']
                , temp['category_name']
                , temp['edge_followed_by']['count']
                , temp['external_url']
                , temp['business_email']
                , temp['business_phone_number']

            ])
    
    except Exception as e:
        print(f"Error processing user {user}: {e}")
        continue
        
    time.sleep(3)


Processing users: 100%|█████████████████████████| 10/10 [00:35<00:00,  3.51s/it]


In [49]:
df_base_account_list = pd.DataFrame(base_account_list, columns = base_account_col)
df_base_account_list.head()

,id,username,full_name,biography_desc,category_name,followers,external_url,business_email,business_phone_number
0,7316146754,officialdonat_bahagia,DONAT BAHAGIA,DONAT BIKIN BAHAGIA | 70 VARIANT RASA | BEBAS ...,Food & beverage,151124,http://www.donatbahagia.id/,None,None
1,36403582263,sagara.homemade,Donat 🍩 Choco Cheese Strip🍫🧀,❌Temporarily Closed❌\nExclusively made for pre...,Food & beverage,27,None,None,None
2,49054349451,dobul.donut,Donat Gembul,Donat lezat 🍩 dari PT. Trinojaya Alenson 🏘\nPr...,Food & beverage,165,None,None,None
3,5877763021,sera.treats,Hampers Lebaran Rancaekek,"📍 Rancaekek, Kab Bandung\n• Tidak Ready Stock\...",Food & beverage,456,https://wa.me/message/6GVZWFEFIIIHB1,None,None
4,53459835779,donusaurus,Donat Kentang Pontianak,"Donat Kentang Premium\n🏡 Pontianak (Serdam, Ko...",None,765,https://shopee.co.id/universal-link/now-food/s...,None,None


# Fetching Suggested Account

In [61]:
def fecthing_suggested_account(userid, cookies, headers, query_id):
    import requests

    cookies = cookies

    headers = headers

    params = {
        'user_id': str(userid),
        'include_chaining': 'false',
        'include_reel': 'true',
        'include_suggested_users': 'false',
        'include_logged_out_extras': 'false',
        'include_live_status': 'false',
        'include_highlight_reels': 'true',
    }
    
    params['query_id'] = query_id['query_id']

    response = requests.get('https://www.instagram.com/graphql/query/', params=params, cookies=cookies, headers=headers)
    
    # return json.loads(response.text)['data']['user']['edge_chaining']['edges'][0]['node']['username']

    return json.loads(response.text)['data']['user']['edge_chaining']['edges']
    

In [65]:
from tqdm import tqdm
import time

suggested_username_list = []

for user in tqdm(df_base_account_list['id'], desc = 'Processing base accounts'):
    temp = fecthing_suggested_account(user, cookies, headers, params)
    
    for i in tqdm(range(len(temp)), desc = 'Processing suggested accounts'):
        try:
            suggested_username_list.append(temp[i]['node']['username'])
            
        except Exception as e:
            print(f"Error processing user {i}: {e}")
            continue


Processing base accounts: 100%|█████████████████| 10/10 [00:12<00:00,  1.21s/it]


In [69]:
suggested_username_list[:10]

['madalangu.bakehouse',
 'fudgybro',
 'theparlorhills',
 'bodabarn.bdg',
 'foursis.space',
 'sincerehomemade',
 'brunobakehouse',
 '150coffeegarden',
 'rinpurin.id',
 'soesjendralbdg']

# Getting All Details from Suggested Accounts

In [71]:
finding_base_account_id(suggested_username_list[0], cookies, headers)

{'ai_agent_type': None,
 'biography': 'Made by Soulful Artisan Young Baker\n• 1st #CromBomb in Bandung\n• 11.00-23.00 | During Ramadan\n• Halal-Certified & Fresh-Baked\n• Menu & Location 👇',
 'bio_links': [{'title': '',
   'lynx_url': 'https://l.instagram.com/?u=http%3A%2F%2Flinktr.ee%2Fmadalangu.bakehouse%2F&e=AT1fX79KzW4r7N9Ob9CkKXj8j5bY95EzUVKA8yIIn1lR5AwKfyE3fxE86Z4YLOoxnq7-5dHLiNGGJJXfIsvjAUDMOxfAcboQ0dmQJIj9KLTCfjvXN0YKAw',
   'url': 'http://linktr.ee/madalangu.bakehouse/',
   'link_type': 'external'}],
 'fb_profile_biolink': None,
 'biography_with_entities': {'raw_text': 'Made by Soulful Artisan Young Baker\n• 1st #CromBomb in Bandung\n• 11.00-23.00 | During Ramadan\n• Halal-Certified & Fresh-Baked\n• Menu & Location 👇',
  'entities': [{'user': None, 'hashtag': {'name': 'crombomb'}}]},
 'blocked_by_viewer': False,
 'restricted_by_viewer': False,
 'country_block': False,
 'eimu_id': '17842211081251460',
 'external_url': 'http://linktr.ee/madalangu.bakehouse/',
 'external_url_link

In [72]:
suggested_account_list = []

suggested_account_col = ['id'
                    , 'username'
                    , 'full_name'
                    , 'biography_desc'
                    , 'category_name'
                    , 'followers'
                    , 'external_url'
                    , 'business_email'
                    , 'business_phone_number'        
                   ]

# for user in suggested_username_list:
for i in tqdm(range(len(suggested_username_list)), desc="Processing items"):
    try:
        temp = finding_base_account_id(suggested_username_list[i], cookies, headers)
        
        suggested_account_list.append(
            [
                temp['id']
                , temp['username']
                , temp['full_name']
                , temp['biography_with_entities']['raw_text']
                , temp['category_name']
                , temp['edge_followed_by']['count']
                , temp['external_url']
                , temp['business_email']
                , temp['business_phone_number']

            ])
        
        if len(suggested_account_list)%50 == 0:
            time.sleep(30)
        else:
            time.sleep(5)
        
    except:
        continue


Processing items: 100%|███████████████████████| 403/403 [40:29<00:00,  6.03s/it]


In [73]:
df_suggested_account_list = pd.DataFrame(suggested_account_list, columns = suggested_account_col)
df_suggested_account_list.head()

,id,username,full_name,biography_desc,category_name,followers,external_url,business_email,business_phone_number
0,37512515459,madalangu.bakehouse,CromBomb & Pastry Bandung,Made by Soulful Artisan Young Baker\n• 1st #Cr...,Dessert Shop,21660,http://linktr.ee/madalangu.bakehouse/,None,None
1,8619223614,fudgybro,FUDGYBRO! BROWNIES 🇲🇨,FUDGIEST & TASTIEST BRONIE YOU'LL EVER TASTE\n...,Food & beverage,76697,https://linktr.ee/fudgybro,None,None
2,2162753622,theparlorhills,The Parlor Hills Bandung,RAMADHAN OPEN HOUR✨\nCAFE :\nSen-Jum : 12.00 -...,Recreation Spot,54671,https://s.id/linkinformasitheparlorhills,None,None
3,50256073795,bodabarn.bdg,Restaurant || Event & Wedding Venue Bandung,Weekdays 12.00-20.00\nWeekends 12.00-20.00\nS...,Restaurant,14241,https://linktr.ee/bodabarndeprovence,None,None
4,11161646368,foursis.space,FOURSIS Cafe & Space,"Sip, spark, and settle.\n📍Jalan Dipatikukur No...",Cafe,4564,https://drive.google.com/drive/u/6/folders/1vz...,None,None


In [74]:
df_suggested_account_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403 entries, 0 to 402
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     403 non-null    object
 1   username               403 non-null    object
 2   full_name              403 non-null    object
 3   biography_desc         403 non-null    object
 4   category_name          323 non-null    object
 5   followers              403 non-null    int64 
 6   external_url           268 non-null    object
 7   business_email         0 non-null      object
 8   business_phone_number  0 non-null      object
dtypes: int64(1), object(8)
memory usage: 28.5+ KB
